In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

try:
    ram_info = psutil.virtual_memory()
    print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
    print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
    print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
    print(f"Percentage Usage Of RAM: {ram_info.percent}%")
    print(f"CPU Cores: {os.cpu_count()}")
    print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
except:
    print("RAM and CPU info not available on this system")

Total RAM: 15.47 GB
Available RAM: 12.25 GB
Used RAM: 2.91 GB
Percentage Usage Of RAM: 20.8%
CPU Cores: 4
CPU Speed: 2.5000 GHz


In [3]:
# import requirements
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from regression import Regression

In [4]:
# get the data
unemployment = pd.read_csv("unemployment.csv")
unemployment = unemployment.sort_values(by=[
    "Date", 
    "Area Type ", 
    "Area Name ", 
    "Seasonally Adjusted (Y/N) ", 
    "Status (Preliminary / Final) ",
]).reset_index(drop=True)
unemployment = unemployment.head(int(0.5 * unemployment.shape[0]))  # only use 50% of the data

In [5]:
# split up the data into training and testing
y = unemployment[["Unemployment Rate "]]
X = unemployment.drop(columns=["Unemployment Rate ", "Employment ", "Unemployment ", "Year ", "Month"])
trainX = X.head(int(0.8 * X.shape[0]))
trainy = y.head(int(0.8 * y.shape[0]))
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [6]:
# build the model
print("\n---- Unemployment Regression Analysis ----\n")
model = Regression(name="Unemployment Regression Analysis", frac=0.8)  # only use 80% of the data for preprocessing
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.fit(trainX, trainy)  # build the machine learning pipeline
    predictions = model.predict(testX)


---- Unemployment Regression Analysis ----

1/6) Model Training
> Renaming Features
> Extracting Time Features
> Transforming Categorical Features
> Renaming Features
> Filling In Missing Values
> Removing Constant Features
> Scaling Features
> Selecting Features
> Computing Atwoood Numbers
> Binning Features
> Computing Reciprocals
> Computing Interactions
> Removing Constant Features
> Selecting Features
> Transforming The Training Data
> Training XGBoost
2.93 Minutes
2/6) Model Performance
> Transforming The Testing Data
10.03 Seconds
3/6) Model Deployment
> Transforming All The Data
> Training XGBoost
52.1 Seconds
4/6) Model Indicators
0.15 Seconds
5/6) Model Prediction
> Transforming The New Data
4.28 Seconds
6/6) Model Monitoring
6.47 Seconds


In [7]:
# model diagnostics
print("Model Diagnostics:")
print(f"> RMSE: {model.rmse}")
print(f"> R2: {model.r2}")
print(f"> In Control: {model.in_control}")
print("> Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10]):
    print(f"{i+1}. {indicator}")
print("> Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10]):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Diagnostics:
> RMSE: 0.06141041107044169
> R2: 0.2429078996483898
> In Control: 96.69%
> Model Indicators:
1. Area_Name__Prattville_CDP
2. Date_01/01/2011
3. Date_02/01/2011*1/Labor_Force
4. Area_Name__Pajaro_CDP
5. Area_Name__Shoshone_CDP
6. Area_Name__Canyondam_CDP
7. Area_Name__Prattville_CDP*Labor_Force
8. Area_Name__Canyondam_CDP*Labor_Force
9. Area_Name__El_Centro,_CA_MSA
10. Area_Name__Chilcoot_Vinton_CDP*Labor_Force
> Feature Drift:
1. Area_Type__Sub-County_Place*Date_01/01/2016
2. Area_Type__Sub-County_Place*Date_03/01/2016
3. Area_Type__Sub-County_Place*Date_01/01/2013
4. Date_03/01/2012*Area_Type__Sub-County_Place
5. Area_Type__Sub-County_Place*Date_01/01/2022
6. Area_Type__Sub-County_Place*Date_01/01/2023
7. Area_Type__Sub-County_Place*Date_01/01/2019
8. Area_Type__Sub-County_Place*Date_03/01/2022
9. Area_Type__Sub-County_Place*Date_03/01/2021
10. Area_Type__Sub-County_Place*Date_01/01/2010


In [8]:
# score the model
rmse = mean_squared_error(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    squared=False,
)
r2 = r2_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 0.06082342631705329
R2: 0.23461296802128107


In [9]:
# save the machine learning pipeline
model.dump()

In [10]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The New Data
> Training XGBoost
43.22 Seconds
